2023/03/03<br>
This code is for concatenating images after prediction by UNet.<br>

In [1]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.filters import threshold_otsu

In [2]:
#set path
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
pred_im_path=os.path.join(npz_path, "pred_im")
pred_im_conc_path=os.path.join(npz_path, "pred_im_concatenation")
fig_save_path=os.path.join(current_path, "fig_save", "pred_concatenation")

for path in [pred_im_path, fig_save_path]:
    if os.path.exists(path)==False:
        os.makedirs(path)

im_size=512
im_step=256

#find target files
target_files=[]
for curdir, _, files in os.walk(pred_im_path):
    if len(files)>0:
        for file in files:
            target_file=os.path.join(curdir, file)
            target_files.append(target_file)

load all images and concatenate them species by species 

In [5]:
for target_file in tqdm(target_files):
    #load npz
    npz=np.load(target_file, allow_pickle=True)
    pred_im=npz["pred_im"]
    ori_im=npz["ori_im"]
    angle_im=npz["angle_im"]
    height_num=np.int(npz["height_num"])
    width_num=np.int(npz["width_num"])
    
    #set matrices
    result_mat=np.zeros((im_step*height_num+im_size, im_step*width_num+im_size))
    MFA_ori_mat=np.zeros((im_step*height_num+im_size, im_step*width_num+im_size))
    angle_mat=np.zeros((im_step*height_num+im_size, im_step*width_num+im_size))
    div_mat=np.zeros((im_step*height_num+im_size, im_step*width_num+im_size))
    
    kernel=np.ones((im_size, im_size))
    for i in range(len(pred_im)):
        q, r = divmod(i, width_num+1)
        result_mat[q*im_step:q*im_step+im_size, 
                   r*im_step:r*im_step+im_size]+=pred_im[i][:, :, 0]
        div_mat[q*im_step:q*im_step+im_size, 
                   r*im_step:r*im_step+im_size]+=kernel
        MFA_ori_mat[q*im_step:q*im_step+im_size, 
                   r*im_step:r*im_step+im_size]+=ori_im[i]
        angle_mat[q*im_step:q*im_step+im_size, 
                   r*im_step:r*im_step+im_size]+=angle_im[i]
    
    #normalize results
    result_mat=result_mat/div_mat
    MFA_ori_mat=MFA_ori_mat/div_mat
    angle_mat=angle_mat/div_mat
    MFA_vis_mat= ((MFA_ori_mat-np.min(MFA_ori_mat))/(np.max(MFA_ori_mat)-np.min(MFA_ori_mat)))*255
    
    #binarization
    thres=threshold_otsu(result_mat)
    pred_binary=np.uint8((result_mat>thres)*255)
    
    #save result as npz
    sample_name=os.path.splitext(os.path.basename(target_file))[0]
    np.savez_compressed(os.path.join(pred_im_conc_path, sample_name+".npz"), 
                       MFA=MFA_ori_mat, angle=angle_mat, pred=result_mat, pred_binary=pred_binary)
    
    #save images as figure
    target_save_path=os.path.join(fig_save_path, sample_name)
    if os.path.exists(target_save_path)==False:
        os.makedirs(target_save_path)
    
    #convert boundary image to transparent one for png
    boundary_image_tr=np.zeros((pred_binary.shape[0], pred_binary.shape[1], 4), np.uint8)
    boundary_image_tr[:,:,2]=pred_binary
    boundary_image_tr[:,:,3] =np.where(np.all(boundary_image_tr == 0, axis=-1), 0, 255) 
    cv2.imwrite(os.path.join(target_save_path, sample_name+".png"), boundary_image_tr)
    cv2.imwrite(os.path.join(target_save_path, sample_name+"_ori.png"), MFA_vis_mat)

100%|██████████| 32/32 [09:26<00:00, 16.35s/it]
